In [40]:
import numpy as np
import tensorflow as tf

In [41]:
%run 11-single_embedding_transformer.ipynb

18887
6
(55698, 79)
(55698, 79)
(64, 79, 1024)
(64, 79, 1024)
(64, 79)
(64, 79, 18887)


In [42]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [43]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [44]:
def masked_loss(label, pred):
  print(label)
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [45]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [47]:
transformer.fit(batched_dataset,
                epochs=1)

Tensor("IteratorGetNext:1", shape=(128, 79), dtype=int32)
Tensor("IteratorGetNext:1", shape=(128, 79), dtype=int32)
435/435 [==============================] - 2246s 5s/step - loss: 8.3888 - masked_accuracy: 0.0287


In [48]:
transformer.save_weights("./transformer_v3/transformer_v3")

In [56]:
starting_string = "Romeo:"

In [57]:
used_tokenizer = data_object.tokenizer

In [58]:
encoded_input = used_tokenizer.texts_to_sequences([starting_string])[0]
encoded_input

[]

In [59]:
inner_token_list = [encoded_input]
output_generated = ""
iter_thing = 1

In [60]:
inner_input = tf.convert_to_tensor(inner_token_list) 
inner_input

<tf.Tensor: shape=(1, 0), dtype=float32, numpy=array([], shape=(1, 0), dtype=float32)>

In [65]:
for i in range(1):
    inner_input = tf.convert_to_tensor([pad_list(inner_token_list[0], 80)])
    predictions  = transformer(inner_input)[:, iter_thing, :]
    print(predictions)
    predicted_id = tf.argmax(predictions[:1], axis=-1)
    
    token = predicted_id.numpy()[0]
    print(token)
    inner_token_list[0].append(token)

    text = used_tokenizer.index_word[token]
    output_generated += (' ' + text)
        
    iter_thing += 1
    inner_token_list[0].append(token)
    

print(output_generated)



tf.Tensor([[-3.9220977  5.470049   4.9243655 ... -3.9239202 -4.2347736 -3.8813748]], shape=(1, 18887), dtype=float32)
1
 the the the the the


In [ ]:
transformer.save_weights("./transformer_v3/transformer_v3")